In [1]:
import os

# change the working directory to root of this project
os.chdir("..")

# verify the working directory
os.getcwd()

'/home/abraham-pc/Documents/personal_projects/Multi-Platform_Job_Recommender/backend'

In [2]:
from backend.src.models.embedding_model import model
import pandas as pd
import numpy as np

/home/abraham-pc/Documents/personal_projects/Multi-Platform_Job_Recommender/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sample_jobs = pd.read_csv("./notebooks/sample_jobs.csv")

In [4]:
job = sample_jobs.iloc[0]

In [5]:
full_text = str()

cols = job.keys().tolist()[3:]

for col in cols:
    full_text += str(job[col]) + " "

In [6]:
full_text

"Indeed jobTitle-d1d12124babd4d39 Airport Attendant Kas Ventures Kas Ventures\nLagos Employer\nActive 5 days ago https://ng.indeed.com/company/KAS-VENTURES/jobs/Airport-Attendant-d1d12124babd4d39?fccid=c9d38c2bd423b7d1&vjs=3 Conducting pre-flight safety checks and ensuring the plane is clean.\nDemonstrating safety and emergency measures, such as the use of oxygen masks, seat belts, and exit doors.\nOffering information and support to ensure the safety and comfort of passengers.\nWelcoming passengers, verifying their tickets, and directing them to their seats.\nMaking and serving beverages and food to passengers.\nResponding to passengers' questions about flights, travel routes, and arrival times.\nAttending pre-flight briefings and studying all the details of the flight.\nLeading passengers in case of emergency and giving first aid to passengers if necessary.\nOffering support to passengers with special needs, including children, disabled persons, and elderly individuals.\nPreparing an

In [7]:
embedding = model.encode(full_text)

In [8]:
type(embedding)

numpy.ndarray

In [9]:
emb_list = embedding.tolist() # convert embedding to list

In [10]:
type(emb_list[0])

float

In [12]:
from backend.etl.databases.milvus.milvus_conn import MilvusConn
# from backend.etl.databases.milvus.table_models import schema
from pymilvus import Collection
from uuid import uuid4
conn = MilvusConn()

collection = Collection(name=conn.collection_name, using=conn.session_name, db_name=conn.database_name)
collection.create_index(
            field_name="embedding",
            index_name="emb_idx",
            index_params={
                "metric_type": "L2",
                "index_type": "HNSW",
                "params": {"M": 64, "efConstruction": 80}
            }
)
collection.load()
collection.insert([[str(uuid4())],[emb_list]])
conn.close()